# 天池o2o优惠券使用预测比赛解析（初级）

**赛题链接：**

[天池o2o优惠券使用预测](https://tianchi.aliyun.com/getStart/introduction.htm?spm=5176.100066.0.0.518433afBqXIKM&raceId=231593)

## 导入相关库

In [1]:
# import libraries necessary for this project
import os, sys, pickle
 
import numpy as np
import pandas as pd
 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
 
import seaborn as sns
 
from datetime import date
 
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler
 
import xgboost as xgb        # pip install xgboost-0.80-py2.py3-none-win_amd64.whl    https://pypi.org/project/xgboost/#files
import lightgbm as lgb       # pip install lightgbm-2.1.2-py2.py3-none-win_amd64.whl  https://pypi.org/project/lightgbm/#files
 
# display for this notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 导入数据

In [2]:
dfoff = pd.read_csv('data/ccf_offline_stage1_train.csv')
dfon = pd.read_csv('data/ccf_online_stage1_train.csv')
dftest = pd.read_csv('data/ccf_offline_stage1_test_revised.csv')
 
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


### 简单统计

简单统计一下用户使用优惠券的情况：

In [3]:
print('有优惠卷，购买商品：%d' % dfoff[(dfoff['Date_received'] != 'null') & (dfoff['Date'] != 'null')].shape[0])
print('有优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'] != 'null') & (dfoff['Date'] == 'null')].shape[0])
print('无优惠卷，购买商品：%d' % dfoff[(dfoff['Date_received'] == 'null') & (dfoff['Date'] != 'null')].shape[0])
print('无优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'] == 'null') & (dfoff['Date'] == 'null')].shape[0])

有优惠卷，购买商品：75382
有优惠卷，未购商品：977900
无优惠卷，购买商品：701602
无优惠卷，未购商品：0


可见，很多人（701602）购买商品却没有使用优惠券，也有很多人（977900）有优惠券但却没有使用，真正使用优惠券购买商品的人（75382）很少！所以，这个比赛的意义就是把优惠券送给真正可能会购买商品的人。

## 特征提取

In [4]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


### 打折率 Discount_rate

In [5]:
print('Discount_rate 类型：\n',dfoff['Discount_rate'].unique())

Discount_rate 类型：
 ['null' '150:20' '20:1' '200:20' '30:5' '50:10' '10:5' '100:10' '200:30'
 '20:5' '30:10' '50:5' '150:10' '100:30' '200:50' '100:50' '300:30' '50:20'
 '0.9' '10:1' '30:1' '0.95' '100:5' '5:1' '100:20' '0.8' '50:1' '200:10'
 '300:20' '100:1' '150:30' '300:50' '20:10' '0.85' '0.6' '150:50' '0.75'
 '0.5' '200:5' '0.7' '30:20' '300:10' '0.2' '50:30' '200:100' '150:5']


打折率分为 3 种情况：

- 'null' 表示没有打折

- [0,1] 表示折扣率

- x:y 表示满x减y

**处理方式：**

- 打折类型：getDiscountType()

- 折扣率：convertRate()

- 满多少：getDiscountMan()

- 减多少：getDiscountJian()

In [6]:
# Convert Discount_rate and Distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)
    
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0
    
def processData(df):
    
    # convert discount_rate
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)
    
    print(df['discount_rate'].unique())
    
    return df

In [7]:
dfoff = processData(dfoff)
dftest = processData(dftest)

[ 1.          0.86666667  0.95        0.9         0.83333333  0.8         0.5
  0.85        0.75        0.66666667  0.93333333  0.7         0.6
  0.96666667  0.98        0.99        0.975       0.33333333  0.2         0.4       ]
[ 0.83333333  0.9         0.96666667  0.8         0.95        0.75        0.98
  0.5         0.86666667  0.6         0.66666667  0.7         0.85
  0.33333333  0.94        0.93333333  0.975       0.99      ]


In [8]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,20
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,1
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,1
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,1


### 距离 Distance

In [9]:
print('Distance 类型：',dfoff['Distance'].unique())

Distance 类型： ['0' '1' 'null' '2' '10' '4' '7' '9' '3' '5' '6' '8']


将距离 str 转为 int。

In [10]:
# convert distance
dfoff['distance'] = dfoff['Distance'].replace('null', -1).astype(int)
print(dfoff['distance'].unique())
dftest['distance'] = dftest['Distance'].replace('null', -1).astype(int)
print(dftest['distance'].unique())

[ 0  1 -1  2 10  4  7  9  3  5  6  8]
[ 1 -1  5  2  0 10  3  6  7  4  9  8]


In [11]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian,distance
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,20,1
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,1,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,1,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,1,0


### 领劵日期 Date_received

In [12]:
date_received = dfoff['Date_received'].unique()
date_received = sorted(date_received[date_received != 'null'])

date_buy = dfoff['Date'].unique()
date_buy = sorted(date_buy[date_buy != 'null'])

print('优惠卷收到日期从',date_received[0],'到',date_received[-1])
print('消费日期从',date_buy[0],'到',date_buy[-1])

优惠卷收到日期从 20160101 到 20160615
消费日期从 20160101 到 20160630


**关于领劵日期的特征：**

- weekday : {null, 1, 2, 3, 4, 5, 6, 7}

- weekday_type : {1, 0}（周六和周日为1，其他为0）

- Weekday_1 : {1, 0, 0, 0, 0, 0, 0}

- Weekday_2 : {0, 1, 0, 0, 0, 0, 0}

- Weekday_3 : {0, 0, 1, 0, 0, 0, 0}

- Weekday_4 : {0, 0, 0, 1, 0, 0, 0}

- Weekday_5 : {0, 0, 0, 0, 1, 0, 0}

- Weekday_6 : {0, 0, 0, 0, 0, 1, 0}

- Weekday_7 : {0, 0, 0, 0, 0, 0, 1}

In [13]:
def getWeekday(row):
    if row == 'null':
        return row
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

dfoff['weekday'] = dfoff['Date_received'].astype(str).apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].astype(str).apply(getWeekday)

# weekday_type :  周六和周日为1，其他为0
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x: 1 if x in [6,7] else 0)
dftest['weekday_type'] = dftest['weekday'].apply(lambda x: 1 if x in [6,7] else 0)

# change weekday to one-hot encoding 
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
#print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace('null', np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace('null', np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

In [14]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,distance,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,...,0,null,0,0,0,0,0,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,1,6,1,0,0,0,0,0,1,0
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,...,0,3,0,0,0,1,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,...,0,6,1,0,0,0,0,0,1,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,0,1,0,1,0,0,0,0,0,0


### 所有特征：

- discount_rate

- discount_type

- discount_man

- discount_jian

- distance

- weekday

- weekday_type

- weekday_1

- weekday_2

- weekday_3

- weekday_4

- weekday_5

- weekday_6

- weekday_7

## 标签标注

三种情况：

- Date_received == 'null'：表示没有领到优惠券，无需考虑，y = -1

- (Date_received != 'null') & (Date != 'null') & (Date - Date_received <= 15)：表示领取优惠券且在15天内使用，即正样本，y = 1

- (Date_received != 'null') & ((Date == 'null') | (Date - Date_received > 15))：表示领取优惠券未在在15天内使用，即负样本，y = 0

定义标签备注函数：

In [15]:
def label(row):
    if row['Date_received'] == 'null':
        return -1
    if row['Date'] != 'null':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff['label'] = dfoff.apply(label, axis=1)

In [16]:
print(dfoff['label'].value_counts())

 0    988887
-1    701602
 1     64395
Name: label, dtype: int64


In [17]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,...,null,0,0,0,0,0,0,0,0,-1
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,6,1,0,0,0,0,0,1,0,0
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,...,3,0,0,0,1,0,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,...,6,1,0,0,0,0,0,1,0,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,1,0,1,0,0,0,0,0,0,0


## 建立线性模型 SGDClassifier

- 使用上面提取的14个特征。

- 训练集：20160101-20160515；验证集：20160516-20160615。

- 用线性模型 SGDClassifier

### 划分训练集/验证集

In [18]:
# data split
df = dfoff[dfoff['label'] != -1].copy()
train = df[(df['Date_received'] < '20160516')].copy()
valid = df[(df['Date_received'] >= '20160516') & (df['Date_received'] <= '20160615')].copy()
print('Train Set: \n', train['label'].value_counts())
print('Valid Set: \n', valid['label'].value_counts())

Train Set: 
 0    759172
1     41524
Name: label, dtype: int64
Valid Set: 
 0    229715
1     22871
Name: label, dtype: int64


### 特征数量

In [19]:
# feature
original_feature = ['discount_rate','discount_type','discount_man', 'discount_jian','distance', 'weekday', 'weekday_type'] + weekdaycols
print('共有特征：',len(original_feature),'个')
print(original_feature)

共有特征： 14 个
['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


### 建立模型

In [20]:
def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log',  # loss function: logistic regression
        penalty='elasticnet', # L1 & L2
        fit_intercept=True,  # 是否存在截距，默认存在
        max_iter=100, 
        shuffle=True,  # Whether or not the training data should be shuffled after each epoch
        n_jobs=1, # The number of processors to use
        class_weight=None) # Weights associated with classes. If not given, all classes are supposed to have weight one.
 
    # 管道机制使得参数集在新数据集（比如测试集）上的重复使用，管道机制实现了对全部步骤的流式化封装和管理。
    model = Pipeline(steps=[
        ('ss', StandardScaler()), # transformer
        ('en', classifier())  # estimator
    ])
 
    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }
 
    # StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    # Exhaustive search over specified parameter values for an estimator.
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1,  # -1 means using all processors
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

### 训练

In [21]:
predictors = original_feature
model = check_model(train, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  2.4min finished


### 验证

对验证集中每个优惠券预测的结果计算 AUC，再对所有优惠券的 AUC 求平均。计算 AUC 的时候，如果 label 只有一类，就直接跳过，因为 AUC 无法计算。

In [22]:
# valid predict
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]
valid1.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label,pred_prob
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,1,0,0,0,0,0,1,0,0,0.019557
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,0,1,0,0,0,0,0,0,0,0.101050
6,1439408,2632,8591,20:1,0,20160516,20160613,1,0.950000,20,...,0,1,0,0,0,0,0,0,0,0.101050
9,2029232,450,1532,30:5,0,20160530,null,1,0.833333,30,...,0,1,0,0,0,0,0,0,0,0.096889
10,2029232,6459,12737,20:1,0,20160519,null,1,0.950000,20,...,0,0,0,0,1,0,0,0,0,0.132660


计算 AUC：

In [23]:
# avgAUC calculation
vg = valid1.groupby(['Coupon_id'])
aucs = []
for i in vg:
    tmpdf = i[1] 
    if len(tmpdf['label'].unique()) != 2:
        continue
    fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.532344469452


### 测试

In [24]:
# test prediction for submission
y_test_pred = model.predict_proba(dftest[predictors])
dftest1 = dftest[['User_id','Coupon_id','Date_received']].copy()
dftest1['Probability'] = y_test_pred[:,1]
dftest1.to_csv('submit1.csv', index=False, header=False)
dftest1.head(5)

,User_id,Coupon_id,Date_received,Probability
0,4129537,9983,20160712,0.105114
1,6949378,3429,20160706,0.153999
2,2166529,6928,20160727,0.005537
3,2166529,1808,20160727,0.018767
4,6172162,6500,20160708,0.063588


### 保存模型 & 导入模型

In [25]:
if not os.path.isfile('1_model.pkl'):
    with open('1_model.pkl', 'wb') as f:
        pickle.dump(model, f)
else:
    with open('1_model.pkl', 'rb') as f:
        model = pickle.load(f)

## 优化模型

### 特征提取

通过客户和商户以前的买卖情况，提取各自或者交叉的特征。这里使用20160101到20160515之间的数据提取特征，20160516-20160615的数据作为训练集（划分训练集和验证集）。

In [26]:
feature = dfoff[(dfoff['Date'] < '20160516') | ((dfoff['Date'] == 'null') & (dfoff['Date_received'] < '20160516'))].copy()
data = dfoff[(dfoff['Date_received'] >= '20160516') & (dfoff['Date_received'] <= '20160615')].copy()
print(data['label'].value_counts())

0    229715
1     22871
Name: label, dtype: int64


In [27]:
fdf = feature.copy()

#### 用户 User 特征

u：用户统计

In [28]:
# key of user
u = fdf[['User_id']].copy().drop_duplicates()

u1:用户接收到优惠券的数量

In [29]:
# u_coupon_count : num of coupon received by user
u1 = fdf[fdf['Date_received'] != 'null'][['User_id']].copy()
u1['u_coupon_count'] = 1
u1 = u1.groupby(['User_id'], as_index = False).count()
u1.head()

,User_id,u_coupon_count
0,4,1
1,35,4
2,36,2
3,64,1
4,110,3


u2：用户购买的次数

In [30]:
# u_buy_count : times of user buy offline (with or without coupon)
u2 = fdf[fdf['Date'] != 'null'][['User_id']].copy()
u2['u_buy_count'] = 1
u2 = u2.groupby(['User_id'], as_index = False).count()
u2.head()

,User_id,u_buy_count
0,165,10
1,184,1
2,215,1
3,285,1
4,315,3


u3：用户使用优惠券购买的次数

In [31]:
# u_buy_with_coupon : times of user buy offline (with coupon)
u3 = fdf[((fdf['Date'] != 'null') & (fdf['Date_received'] != 'null'))][['User_id']].copy()
u3['u_buy_with_coupon'] = 1
u3 = u3.groupby(['User_id'], as_index = False).count()
u3.head()

,User_id,u_buy_with_coupon
0,184,1
1,417,1
2,687,2
3,696,1
4,947,1


u4：用户购买的商家个数

In [32]:
# u_merchant_count : num of merchant user bought from
u4 = fdf[fdf['Date'] != 'null'][['User_id', 'Merchant_id']].copy()
u4.drop_duplicates(inplace = True)
u4 = u4.groupby(['User_id'], as_index = False).count()
u4.rename(columns = {'Merchant_id':'u_merchant_count'}, inplace = True)
u4.head()

,User_id,u_merchant_count
0,165,2
1,184,1
2,215,1
3,285,1
4,315,2


u5：用户使用优惠券购买商品距离商店的最小距离

u6：用户使用优惠券购买商品距离商店的最大距离

u7：用户使用优惠券购买商品距离商店的平均距离

u8：用户使用优惠券购买商品距离商店的中位数距离

In [33]:
# u_min_distance
utmp = fdf[(fdf['Date'] != 'null') & (fdf['Date_received'] != 'null')][['User_id', 'distance']].copy()
utmp.replace(-1, np.nan, inplace = True)
u5 = utmp.groupby(['User_id'], as_index = False).min()
u5.rename(columns = {'distance':'u_min_distance'}, inplace = True)
u6 = utmp.groupby(['User_id'], as_index = False).max()
u6.rename(columns = {'distance':'u_max_distance'}, inplace = True)
u7 = utmp.groupby(['User_id'], as_index = False).mean()
u7.rename(columns = {'distance':'u_mean_distance'}, inplace = True)
u8 = utmp.groupby(['User_id'], as_index = False).median()
u8.rename(columns = {'distance':'u_median_distance'}, inplace = True)
u8.head()

,User_id,u_median_distance
0,184,0.0
1,417,0.0
2,687,NaN
3,696,0.0
4,947,8.0


根据 User_id，将 u1,u2,u3,u4,u5,u6,u7,u8 整合成 user_feature

In [34]:
# merge all the features on key User_id
user_feature = pd.merge(u, u1, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u2, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u3, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u4, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u5, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u6, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u7, on = 'User_id', how = 'left')
user_feature = pd.merge(user_feature, u8, on = 'User_id', how = 'left')

u_use_coupon_rate：对于用户来说，接收到的优惠券使用率

u_buy_with_coupon_rate：用户所有购买行为中使用优惠券占的比例

In [35]:
# calculate rate
user_feature['u_use_coupon_rate'] = user_feature['u_buy_with_coupon'].astype('float')/user_feature['u_coupon_count'].astype('float')
user_feature['u_buy_with_coupon_rate'] = user_feature['u_buy_with_coupon'].astype('float')/user_feature['u_buy_count'].astype('float')
user_feature = user_feature.fillna(0)
user_feature.head()

,User_id,u_coupon_count,u_buy_count,u_buy_with_coupon,u_merchant_count,u_min_distance,u_max_distance,u_mean_distance,u_median_distance,u_use_coupon_rate,u_buy_with_coupon_rate
0,1439408,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1832624,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2029232,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2223968,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,73611,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 商户 Merchant 特征

m：商户统计

In [36]:
# key of merchant
m = fdf[['Merchant_id']].copy().drop_duplicates()

m1：每个商户发放的优惠券数量

In [37]:
# m_coupon_count : num of coupon from merchant
m1 = fdf[fdf['Date_received'] != 'null'][['Merchant_id']].copy()
m1['m_coupon_count'] = 1
m1 = m1.groupby(['Merchant_id'], as_index = False).count()
m1.head()

,Merchant_id,m_coupon_count
0,2,1
1,5,5
2,8,2
3,13,3
4,14,10


m2：商户销售的次数（不考虑是否使用优惠券）

In [38]:
# m_sale_count : num of sale from merchant (with or without coupon)
m2 = fdf[fdf['Date'] != 'null'][['Merchant_id']].copy()
m2['m_sale_count'] = 1
m2 = m2.groupby(['Merchant_id'], as_index = False).count()
m2.head()

,Merchant_id,m_sale_count
0,1,5
1,3,8
2,4,19
3,5,27
4,6,42


m3：商户使用优惠券销售的次数

In [39]:
# m_sale_with_coupon : num of sale from merchant with coupon usage
m3 = fdf[(fdf['Date'] != 'null') & (fdf['Date_received'] != 'null')][['Merchant_id']].copy()
m3['m_sale_with_coupon'] = 1
m3 = m3.groupby(['Merchant_id'], as_index = False).count()
m3.head()

,Merchant_id,m_sale_with_coupon
0,13,1
1,14,1
2,15,11
3,17,3
4,18,2


m4：商家销售的用户个数

In [40]:
# u_merchant_count : num of merchant user bought from
m4 = fdf[fdf['Date'] != 'null'][['User_id', 'Merchant_id']].copy()
m4.drop_duplicates(inplace = True)
m4 = m4.groupby(['Merchant_id'], as_index = False).count()
m4.rename(columns = {'User_id':'m_user_count'}, inplace = True)
m4.head()

,Merchant_id,m_user_count
0,1,2
1,3,8
2,4,2
3,5,9
4,6,8


m5：商户使用优惠券销售商品距离用户的最小距离

m6：商户使用优惠券销售商品距离用户的最大距离

m7：商户使用优惠券销售商品距离用户的平均距离

m8：商户使用优惠券销售商品距离用户的中位数距离

In [41]:
# m_min_distance
mtmp = fdf[(fdf['Date'] != 'null') & (fdf['Date_received'] != 'null')][['Merchant_id', 'distance']].copy()
mtmp.replace(-1, np.nan, inplace = True)
m5 = mtmp.groupby(['Merchant_id'], as_index = False).min()
m5.rename(columns = {'distance':'m_min_distance'}, inplace = True)
m6 = mtmp.groupby(['Merchant_id'], as_index = False).max()
m6.rename(columns = {'distance':'m_max_distance'}, inplace = True)
m7 = mtmp.groupby(['Merchant_id'], as_index = False).mean()
m7.rename(columns = {'distance':'m_mean_distance'}, inplace = True)
m8 = mtmp.groupby(['Merchant_id'], as_index = False).median()
m8.rename(columns = {'distance':'m_median_distance'}, inplace = True)
m8.head()

,Merchant_id,m_median_distance
0,13,0.0
1,14,0.0
2,15,0.0
3,17,0.0
4,18,0.0


根据 Merchant_id，将 m1,m2,m3,m4,m5,m6,m7,m8 整合成 merchant_feature

In [42]:
merchant_feature = pd.merge(m, m1, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m2, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m3, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m4, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m5, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m6, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m7, on = 'Merchant_id', how = 'left')
merchant_feature = pd.merge(merchant_feature, m8, on = 'Merchant_id', how = 'left')
merchant_feature = merchant_feature.fillna(0)
merchant_feature.head()

,Merchant_id,m_coupon_count,m_sale_count,m_sale_with_coupon,m_user_count,m_min_distance,m_max_distance,m_mean_distance,m_median_distance
0,2632,35.0,18.0,3.0,8.0,1.0,1.0,1.000000,1.0
1,3381,122834.0,18080.0,2487.0,11006.0,0.0,10.0,1.652429,1.0
2,2099,16824.0,7227.0,1705.0,2751.0,0.0,10.0,0.968072,0.0
3,1569,33492.0,896.0,135.0,683.0,0.0,10.0,2.260163,1.0
4,4833,8321.0,636.0,116.0,270.0,0.0,10.0,3.037736,2.0


m_coupon_use_rate： 对于商家来说，发放的优惠卷使用率

m_sale_with_coupon_rate：商家所有销售行为中使用优惠卷占的比例

In [43]:
merchant_feature['m_coupon_use_rate'] = merchant_feature['m_sale_with_coupon'].astype('float')/merchant_feature['m_coupon_count'].astype('float')
merchant_feature['m_sale_with_coupon_rate'] = merchant_feature['m_sale_with_coupon'].astype('float')/merchant_feature['m_sale_count'].astype('float')
merchant_feature = merchant_feature.fillna(0)
merchant_feature.head()

,Merchant_id,m_coupon_count,m_sale_count,m_sale_with_coupon,m_user_count,m_min_distance,m_max_distance,m_mean_distance,m_median_distance,m_coupon_use_rate,m_sale_with_coupon_rate
0,2632,35.0,18.0,3.0,8.0,1.0,1.0,1.000000,1.0,0.085714,0.166667
1,3381,122834.0,18080.0,2487.0,11006.0,0.0,10.0,1.652429,1.0,0.020247,0.137555
2,2099,16824.0,7227.0,1705.0,2751.0,0.0,10.0,0.968072,0.0,0.101343,0.235921
3,1569,33492.0,896.0,135.0,683.0,0.0,10.0,2.260163,1.0,0.004031,0.150670
4,4833,8321.0,636.0,116.0,270.0,0.0,10.0,3.037736,2.0,0.013941,0.182390


#### 用户和商户 User & Merchant 交叉特征

um1：用户和商户对统计

In [44]:
# key of user and merchant
um = fdf[['User_id', 'Merchant_id']].copy().drop_duplicates()

In [45]:
um1 = fdf[['User_id', 'Merchant_id']].copy()
um1['um_count'] = 1
um1 = um1.groupby(['User_id', 'Merchant_id'], as_index = False).count()
um1.head()

,User_id,Merchant_id,um_count
0,4,1433,1
1,35,3381,4
2,36,1041,1
3,36,5717,1
4,64,2146,1


um2：每个用户商户对交易统计

In [46]:
um2 = fdf[fdf['Date'] != 'null'][['User_id', 'Merchant_id']].copy()
um2['um_buy_count'] = 1
um2 = um2.groupby(['User_id', 'Merchant_id'], as_index = False).count()
um2.head()

,User_id,Merchant_id,um_buy_count
0,165,2934,6
1,165,4195,4
2,184,3381,1
3,215,129,1
4,285,450,1


um3：每个用户商户对发放优惠券的统计

In [47]:
um3 = fdf[fdf['Date_received'] != 'null'][['User_id', 'Merchant_id']].copy()
um3['um_coupon_count'] = 1
um3 = um3.groupby(['User_id', 'Merchant_id'], as_index = False).count()
um3.head()

,User_id,Merchant_id,um_coupon_count
0,4,1433,1
1,35,3381,4
2,36,1041,1
3,36,5717,1
4,64,2146,1


um4：每个用户商户对使用优惠卷的交易行为的统计

In [48]:
um4 = fdf[(fdf['Date_received'] != 'null') & (fdf['Date'] != 'null')][['User_id', 'Merchant_id']].copy()
um4['um_buy_with_coupon'] = 1
um4 = um4.groupby(['User_id', 'Merchant_id'], as_index = False).count()
um4.head()

,User_id,Merchant_id,um_buy_with_coupon
0,184,3381,1
1,417,775,1
2,687,6454,1
3,687,8594,1
4,696,4195,1


根据 User_id 和 Merchant_id，将 um,um1,um2,um3,um4 整合成 user_merchant_feature

In [49]:
# merge all user merchant 
user_merchant_feature = pd.merge(um, um1, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = pd.merge(user_merchant_feature, um2, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = pd.merge(user_merchant_feature, um3, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = pd.merge(user_merchant_feature, um4, on = ['User_id','Merchant_id'], how = 'left')
user_merchant_feature = user_merchant_feature.fillna(0)

um_buy_rate：每个用户商户对交易行为占所有用户商户对的比例

um_coupon_use_rate：使用优惠卷的交易行为占所有用户商户对发放优惠卷的比例

um_buy_with_coupon_rate：使用优惠卷的交易行为占所有用户商户对交易行为的比例

In [50]:
user_merchant_feature['um_buy_rate'] = user_merchant_feature['um_buy_count'].astype('float')/user_merchant_feature['um_count'].astype('float')
user_merchant_feature['um_coupon_use_rate'] = user_merchant_feature['um_buy_with_coupon'].astype('float')/user_merchant_feature['um_coupon_count'].astype('float')
user_merchant_feature['um_buy_with_coupon_rate'] = user_merchant_feature['um_buy_with_coupon'].astype('float')/user_merchant_feature['um_buy_count'].astype('float')
user_merchant_feature = user_merchant_feature.fillna(0)
user_merchant_feature.head()

,User_id,Merchant_id,um_count,um_buy_count,um_coupon_count,um_buy_with_coupon,um_buy_rate,um_coupon_use_rate,um_buy_with_coupon_rate
0,1439408,2632,3,1.0,2.0,0.0,0.333333,0.0,0.0
1,1832624,3381,1,0.0,1.0,0.0,0.000000,0.0,0.0
2,2029232,3381,1,0.0,1.0,0.0,0.000000,0.0,0.0
3,2223968,3381,1,0.0,1.0,0.0,0.000000,0.0,0.0
4,73611,2099,1,0.0,1.0,0.0,0.000000,0.0,0.0


#### 将 user_feature, merchant_feature, user_merchant_feature 放入训练集和测试集中

In [51]:
# add user_feature, merchant_feature, user_merchant_feature to train data 
data2 = pd.merge(data, user_feature, on = 'User_id', how = 'left').fillna(0)
data3 = pd.merge(data2, merchant_feature, on = 'Merchant_id', how = 'left').fillna(0)
data4 = pd.merge(data3, user_merchant_feature, on = ['User_id','Merchant_id'], how = 'left').fillna(0)
train = data4.copy()

# add user_feature, merchant_feature, user_merchant_feature to test data 
data2 = pd.merge(dftest, user_feature, on = 'User_id', how = 'left').fillna(0)
data3 = pd.merge(data2, merchant_feature, on = 'Merchant_id', how = 'left').fillna(0)
data4 = pd.merge(data3, user_merchant_feature, on = ['User_id','Merchant_id'], how = 'left').fillna(0)
test = data4.copy()

In [52]:
train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,m_median_distance,m_coupon_use_rate,m_sale_with_coupon_rate,um_count,um_buy_count,um_coupon_count,um_buy_with_coupon,um_buy_rate,um_coupon_use_rate,um_buy_with_coupon_rate
0,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,0.0,0.010323,0.034722,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,1.0,0.085714,0.166667,3.0,1.0,2.0,0.0,0.333333,0.0,0.0
2,1439408,2632,8591,20:1,0,20160516,20160613,1,0.950000,20,...,1.0,0.085714,0.166667,3.0,1.0,2.0,0.0,0.333333,0.0,0.0
3,2029232,450,1532,30:5,0,20160530,null,1,0.833333,30,...,0.0,0.015813,0.079193,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,2029232,6459,12737,20:1,0,20160519,null,1,0.950000,20,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [53]:
test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,discount_type,discount_rate,discount_man,discount_jian,...,m_median_distance,m_coupon_use_rate,m_sale_with_coupon_rate,um_count,um_buy_count,um_coupon_count,um_buy_with_coupon,um_buy_rate,um_coupon_use_rate,um_buy_with_coupon_rate
0,4129537,450,9983,30:5,1,20160712,1,0.833333,30,5,...,0.0,0.015813,0.079193,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6949378,1300,3429,30:5,null,20160706,1,0.833333,30,5,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2166529,7113,6928,200:20,5,20160727,1,0.900000,200,20,...,1.0,0.016321,0.022189,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2166529,7113,1808,100:10,5,20160727,1,0.900000,100,10,...,1.0,0.016321,0.022189,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6172162,7605,6500,30:1,2,20160708,1,0.966667,30,1,...,0.0,0.203196,0.119143,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 所有特征

In [54]:
predictors = original_feature + user_feature.columns.tolist()[1:] + \
             merchant_feature.columns.tolist()[1:] + \
             user_merchant_feature.columns.tolist()[2:]
print(len(predictors),predictors)

41 ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7', 'u_coupon_count', 'u_buy_count', 'u_buy_with_coupon', 'u_merchant_count', 'u_min_distance', 'u_max_distance', 'u_mean_distance', 'u_median_distance', 'u_use_coupon_rate', 'u_buy_with_coupon_rate', 'm_coupon_count', 'm_sale_count', 'm_sale_with_coupon', 'm_user_count', 'm_min_distance', 'm_max_distance', 'm_mean_distance', 'm_median_distance', 'm_coupon_use_rate', 'm_sale_with_coupon_rate', 'um_count', 'um_buy_count', 'um_coupon_count', 'um_buy_with_coupon', 'um_buy_rate', 'um_coupon_use_rate', 'um_buy_with_coupon_rate']


### 划分训练集和验证集

In [55]:
trainSub, validSub = train_test_split(train, test_size = 0.2, stratify = train['label'], random_state=100)

### 线性模型 SGDClassifier

#### 建立模型

In [56]:
def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log',  # loss function: logistic regression
        penalty='elasticnet', # L1 & L2
        fit_intercept=True,  # 是否存在截距，默认存在
        max_iter=100, 
        shuffle=True,  # Whether or not the training data should be shuffled after each epoch
        n_jobs=1, # The number of processors to use
        class_weight=None) # Weights associated with classes. If not given, all classes are supposed to have weight one.
 
    # 管道机制使得参数集在新数据集（比如测试集）上的重复使用，管道机制实现了对全部步骤的流式化封装和管理。
    model = Pipeline(steps=[
        ('ss', StandardScaler()), # transformer
        ('en', classifier())  # estimator
    ])
 
    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }
 
    # StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    # Exhaustive search over specified parameter values for an estimator.
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1,  # -1 means using all processors
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

#### 训练

In [57]:
model = check_model(trainSub, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   56.3s finished


#### 验证

对验证集中每个优惠券预测的结果计算 AUC，再对所有优惠券的 AUC 求平均。计算 AUC 的时候，如果 label 只有一类，就直接跳过，因为 AUC 无法计算。

In [58]:
# valid predict
y_valid_pred = model.predict_proba(validSub[predictors])
validSub1 = validSub.copy()
validSub1['pred_prob'] = y_valid_pred[:, 1]
validSub1.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,m_coupon_use_rate,m_sale_with_coupon_rate,um_count,um_buy_count,um_coupon_count,um_buy_with_coupon,um_buy_rate,um_coupon_use_rate,um_buy_with_coupon_rate,pred_prob
166704,2266960,5717,8192,20:5,1,20160525,null,1,0.750000,20,...,0.021105,0.052307,1.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.025477
183019,1748229,4579,12852,5:1,0,20160526,null,1,0.800000,5,...,0.234043,0.027792,1.0,1.0,0.0,0.0,1.000000,0.000000,0.000000,0.174282
134721,2761603,3621,4823,20:5,3,20160606,null,1,0.750000,20,...,0.081267,0.068635,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.041830
10203,6092712,5256,5928,30:5,0,20160521,20160523,1,0.833333,30,...,0.286432,0.240506,49.0,41.0,26.0,18.0,0.836735,0.692308,0.439024,0.662396
209055,5542881,1469,7430,50:20,1,20160521,null,1,0.600000,50,...,0.092346,0.022431,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.016961


计算 AUC：

In [59]:
# avgAUC calculation
vg = validSub1.groupby(['Coupon_id'])
aucs = []
for i in vg:
    tmpdf = i[1] 
    if len(tmpdf['label'].unique()) != 2:
        continue
    fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.616803899905


### 集成模型 LightGBM

#### 训练

In [60]:
model = lgb.LGBMClassifier(
                    learning_rate = 0.01,
                    boosting_type = 'gbdt',
                    objective = 'binary',
                    metric = 'logloss',
                    max_depth = 5,
                    sub_feature = 0.7,
                    num_leaves = 3,
                    colsample_bytree = 0.7,
                    n_estimators = 5000,
                    early_stop = 50,
                    verbose = -1)
model.fit(trainSub[predictors], trainSub['label'])

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        early_stop=50, importance_type='split', learning_rate=0.01,
        max_depth=5, metric='logloss', min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=5000,
        n_jobs=-1, num_leaves=3, objective='binary', random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, sub_feature=0.7,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
        verbose=-1)

#### 验证

对验证集中每个优惠券预测的结果计算 AUC，再对所有优惠券的 AUC 求平均。计算 AUC 的时候，如果 label 只有一类，就直接跳过，因为 AUC 无法计算。

In [61]:
# valid set performance 
y_valid_pred = model.predict_proba(validSub[predictors])
validSub1 = validSub.copy()
validSub1['pred_prob'] = y_valid_pred[:, 1]
validSub1.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,m_coupon_use_rate,m_sale_with_coupon_rate,um_count,um_buy_count,um_coupon_count,um_buy_with_coupon,um_buy_rate,um_coupon_use_rate,um_buy_with_coupon_rate,pred_prob
166704,2266960,5717,8192,20:5,1,20160525,null,1,0.750000,20,...,0.021105,0.052307,1.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.018569
183019,1748229,4579,12852,5:1,0,20160526,null,1,0.800000,5,...,0.234043,0.027792,1.0,1.0,0.0,0.0,1.000000,0.000000,0.000000,0.082810
134721,2761603,3621,4823,20:5,3,20160606,null,1,0.750000,20,...,0.081267,0.068635,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.021860
10203,6092712,5256,5928,30:5,0,20160521,20160523,1,0.833333,30,...,0.286432,0.240506,49.0,41.0,26.0,18.0,0.836735,0.692308,0.439024,0.760965
209055,5542881,1469,7430,50:20,1,20160521,null,1,0.600000,50,...,0.092346,0.022431,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.013496


计算 AUC：

In [62]:
vg = validSub1.groupby(['Coupon_id'])
aucs = []
for i in vg:
    tmpdf = i[1] 
    if len(tmpdf['label'].unique()) != 2:
        continue
    fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.630400363653


### 测试并提交成绩

In [63]:
# test prediction for submission
y_test_pred = model.predict_proba(test[predictors])
submit = test[['User_id','Coupon_id','Date_received']].copy()
submit['label'] = y_test_pred[:,1]
submit.to_csv('submit2.csv', index=False, header=False)
submit.head()

,User_id,Coupon_id,Date_received,label
0,4129537,9983,20160712,0.014623
1,6949378,3429,20160706,0.143826
2,2166529,6928,20160727,0.006144
3,2166529,1808,20160727,0.006103
4,6172162,6500,20160708,0.091049


### 保存模型 & 导入模型

In [64]:
if not os.path.isfile('2_model.pkl'):
    with open('2_model.pkl', 'wb') as f:
        pickle.dump(model, f)
else:
    with open('2_model.pkl', 'rb') as f:
        model = pickle.load(f)

## 优化模型...

- **特征工程**

- **机器学习算法**

- **模型集成**

## 参考代码

[比赛第一名代码与解析](https://github.com/wepe/O2O-Coupon-Usage-Forecast)